# Experiment and path specification

In [ ]:
# Specify the experiment for analysis
# experiment='cxic00318'
# runNumber = 123
# experiment='cxix40218'
experiment='cxilu9218'
# runNumber=31 # includes cspad info
# runNumber=32 # includes cspad info
runNumber=33 # includes cspad info
runNumber=29 # includes cspad info
runNumber=28 # includes cspad info
runNumber=27 # includes cspad info
runNumber=26 # includes cspad info
runNumber=14 # includes cspad info
runNumber=13 # includes cspad info
runNumber=21 # includes cspad info
runNumber=37 # includes cspad info
runNumber=39 # includes cspad info
runNumber=49 # includes cspad info
runNumber=50 # includes cspad info
runNumber=14 # includes cspad info
runNumber=64 # includes cspad info


# Set the installation and output path
import os
os.environ['INSTALLPATH']= '/cds/home/m/mrware/TRXS-dev/TRXS-Run18v3'
os.environ['OUTPUTPATH']= '/cds/data/psdm/%s/%s/scratch' % (experiment[0:3],experiment)

# Import Libraries

In [ ]:
# Magic iPython command to enable plotting
%matplotlib inline

# Load in the pythonBatchMagic library
import sys
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *

# Determine current user
currentUser, error = unixCMD("echo $USER")
currentUser = currentUser.strip()
print(currentUser)
os.environ['RESULTSPATH']= ('/cds/data/psdm/%s/%s/results/%s' % (experiment[0:3],experiment,currentUser)).strip()
if not os.path.exists(os.environ['RESULTSPATH']):
    os.mkdir(os.environ['RESULTSPATH'])

# Leveraging the batch queue to quickly grab point data

## Loading libraries

In [ ]:
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from LCLSdefault import *

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsLibraries')
from picklez import *

# Load in the get data library
from dataAnalysis import *

# Load in the batch library for lcls
from lclsBatch import *

## Load point data

In [ ]:
runNumber = 18
detArrays = load_obj(os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber)

In [ ]:
detArrays['ttfltpos'].shape

In [ ]:
detArrays['stageencoder'].shape

In [ ]:
detArrays['rois'].shape

In [ ]:
detArrays['laserOn'].shape

In [ ]:
detArrays['xrayOn'].shape

## Plot histogram of result

In [ ]:
fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['stageencoder'], bins=100,normed=False, alpha=0.5);
print(detArrays['xrayEnergy'].shape)
# plt.savefig('runfolder/hist-xrayenergy')

In [ ]:
adu = np.arange(200) * 10
plt.semilogy(adu, np.nansum(detArrays['photonHist'][detArrays['xrayOn']],0));
plt.xlabel('adu')
plt.ylabel('counts')
# plt.xlim(250,500)

In [ ]:


fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['gasPressure'], bins=100,normed=False, alpha=0.5);
# print(detArrays['xrayEnergy'].shape)
# plt.savefig('runfolder/hist-xrayenergy')

In [ ]:
detArrays['rois']

In [ ]:


fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['rois'][:,15], bins=100,normed=False, alpha=0.5);
# print(detArrays['xrayEnergy'].shape)
# plt.savefig('runfolder/hist-xrayenergy')

In [ ]:
x = np.linspace(0,1.5e5, 20)
plt.plot(x,np.nanmean(detArrays['rois'][1:],0))
plt.xlabel('pixel pos')
plt.ylabel('mean adu')


In [ ]:
plt.scatter(detArrays['rois'][detArrays['xrayOn'].astype(bool),3],detArrays['rois'][detArrays['xrayOn'].astype(bool),10])

# Look at ROIs vs. stage position

In [ ]:
goodIdx = detArrays['laserOn'] & detArrays['xrayOn'] & ( detArrays['rois'][:,10] > 0 )
pos = detArrays['stageencoder'][goodIdx] 
ttpos = detArrays['ttfltpos'][goodIdx]
goodRois = detArrays['rois'][goodIdx,:]
print(goodRois.shape)
roi1 = np.sum(goodRois, -1)
plotMe = ((goodRois.T)/(roi1.T)).T
print(plotMe.shape,pos.shape)
plt.scatter(pos,  plotMe[:,14]);

roi1 = np.nanmean(plotMe[pos>56.35,:],0  )
roi2 = np.nanmean(plotMe[pos<56.35,:],0)
print(roi1.shape)
for el in (roi1-roi2):
    print('%.7f'%el)

In [ ]:
plt.plot(np.sort(np.unique(pos,axis=None)))

In [ ]:
db = 1e-3
bins = np.unique(pos) - db
bins = np.append(bins, bins[-1]+ 2*db)
nb = bins.size
nr = 20

def createBinsFromCenters(centers):
    bins = []
    nc = centers.size
    for idx,c in enumerate(centers):
        if idx == 0:
            dc = np.abs( c - centers[idx])/2.
            bins.append(c-dc)
            bins.append(c+dc)
        elif idx == nc-1:
            dc = np.abs( c - centers[idx-1])/2.
            bins.append(c+dc)
        else:
            dc = np.abs( c - centers[idx+1])/2.
            bins.append(c+dc)
#         print(dc)
    return np.array(bins)

def determineGoodCenters( pos ):
    upos = np.unique(pos)
    gpos = []
    for idx,up in enumerate(upos):
        c = np.sum(upos == up)
        if c>10:
            gpos.append()
        
bins = createBinsFromCenters(  np.sort(np.unique(pos,axis=None)).flatten())
# print(bins)

weightMe = ((goodRois.T)/(roi1.T)).T
# weightMe = (goodRois)
print(pos.size)
print(weightMe.shape)
radialHist = np.zeros((nb-1,nr))
radialAvg = np.zeros((nb-1,nr))
counts,edges = np.histogram( pos,bins=bins)
for ir in range(nr):

    radialHist[:,ir],edges = np.histogram( pos,bins=bins, weights=weightMe[:,ir])
    radialAvg[:,ir] = radialHist[:,ir] / counts
    
# print(counts)

In [ ]:
# print(plot2d.shape)

In [ ]:
np.sum(counts)

In [ ]:
centers = np.sort(np.unique(pos,axis=None)).flatten()

plot2d= (radialAvg) #/avgAll
avgAll = np.mean( plot2d[counts>100,:][-3:,:], 0 )
plot2d = (plot2d-avgAll) #/avgAll
dv = .002
# dv = .003
# dv = .001
# qs = x/ 31578.94736842 * 3.5
plt.pcolormesh(x, centers[counts>100], plot2d[counts>100,:], vmin = -dv, vmax = dv )
plt.colorbar()
plt.xlabel('pixel pos (um)')
plt.ylabel('delay (mm)')
plt.title('I - I(delay<56.35) for run %d' %runNumber)
print(x)

In [ ]:
plot2d

# Repeat with timetool correction

In [ ]:
ttpoly = [2.95684259e-06, -1.43969413e-03]
def ttcorr(ttpos,ttpoly):
    return ttpoly[0]*ttpos+ttpoly[1]
truepos = -(pos-56.35) / (3e-4)  + ttcorr(ttpos,ttpoly)*1.0e6
posfs = -(pos-56.35) / (3e-4) 

plt.hist(truepos,bins=100);
plt.xlabel('pump-probe delay (fs)')
plt.ylabel('counts')
plt.xlim([-2000,5000])
plt.title('pump-probe delay histogram for run %d' % runNumber)


In [ ]:
db = 1e-3
bins = np.unique(pos) - db
bins = np.append(bins, bins[-1]+ 2*db)
nb = bins.size
nr = 20

def createBinsFromCenters(centers):
    bins = []
    nc = centers.size
    for idx,c in enumerate(centers):
        if idx == 0:
            dc = np.abs( c - centers[idx])/2.
            bins.append(c-dc)
            bins.append(c+dc)
        elif idx == nc-1:
            dc = np.abs( c - centers[idx-1])/2.
            bins.append(c+dc)
        else:
            dc = np.abs( c - centers[idx+1])/2.
            bins.append(c+dc)
#         print(dc)
    return np.array(bins)

def determineGoodCenters( pos ):
    upos = np.unique(pos)
    gpos = []
    for idx,up in enumerate(upos):
        c = np.sum(upos == up)
        if c>10:
            gpos.append()
        
centers = np.sort(np.unique(posfs,axis=None)).flatten()
bins = createBinsFromCenters(  centers )
# print(bins)


weightMe = ((goodRois.T)/(roi1.T)).T

radialHist = np.zeros((nb-1,nr))
radialAvg = np.zeros((nb-1,nr))
counts,edges = np.histogram( truepos,bins=bins)
for ir in range(nr):

    radialHist[:,ir],edges = np.histogram( truepos,bins=bins, weights=weightMe[:,ir])
    radialAvg[:,ir] = radialHist[:,ir] / counts
    
# print(counts)

In [ ]:
plot2d= (radialAvg) #/avgAll
avgAll = np.mean( plot2d[counts>100,:][:2,:], 0 )
plot2d = (plot2d-avgAll) #/avgAll
dv = .001
# dv = .003
# dv = .001
# qs = x/ 31578.94736842 * 3.5
plt.pcolormesh(x, centers[counts>100], plot2d[counts>100,:], vmin = -dv, vmax = dv )
plt.colorbar()
plt.xlabel('pixel pos (um)')
plt.ylabel('delay (fs)')
plt.title('I - I(delay<56.35) for run %d' %runNumber)
# plt.ylim([-500,1000])
print(x)

In [ ]:
plot2d[counts>100,:]

In [ ]:
weightMe.shape